In [1]:
#https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-0-q-learning-with-tables-and-neural-networks-d195264329d0
# Unlike the blog, we will be using Keras in this example

import gym
import numpy as np
from collections import deque
import random

from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

Using TensorFlow backend.
/home/rafaelvcantero/.conda/envs/tf2/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/rafaelvcantero/.conda/envs/tf2/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/rafaelvcantero/.conda/envs/tf2/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/rafaelvc

More info about the Frozen Lake in the documentation: 
https://github.com/openai/gym/blob/master/gym/envs/toy_text/frozen_lake.py

In [2]:
env = gym.make('FrozenLake-v0')
env.render()


SFFF
FHFH
FFFH
HFFG


In [5]:
# Amazing blog post on DQNs which is also where I got the base code for this DQNAgent class. Must read!
# https://keon.io/deep-q-learning/

# Deep Q-learning Agent
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()
        
    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_shape=(self.state_size,), activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))

        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)

        act_values = self.model.predict(np.array([state]))
        return np.argmax(act_values[0])  # returns action
    
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            else:
                target = reward
            target_out = self.model.predict(state)
            target_out[0][action] = target
            self.model.fit(state, target_out, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
#         state, action, reward, next_state, done = zip(*random.sample(self.memory,batch_size))
        
#         future_q = self.model.predict(np.array(next_state))
#         target_out = self.model.predict(np.array(state))
#         for x in range(0,batch_size):
#             if not done[x]:
#                 target = reward[x] + self.gamma * np.amax(future_q[x][0])
#             else:
#                 target = reward[x]
#             target_out[x][action[x]] = target
#         print(np.array(state), target_out)
#         self.model.fit(np.array(state), target_out, epochs=1, verbose=0)
        
#         if self.epsilon > self.epsilon_min:
#             self.epsilon *= self.epsilon_decay

In [6]:
num_episodes = 10
batch_size = 4

# Create the agent
agent = DQNAgent(env.observation_space.n, env.action_space.n)

for episode in range(num_episodes):
    # Reset the env
    pos = env.reset()
    state = np.eye(N=1, M=16, k=pos, dtype=int)[0] # We use the numpy.eye function to create the one-hot vector
    step = 0
    episode_reward = 0
    # We limit the amount of steps to avoid any infinite or long episodes
    while step < 99:
        action = agent.act(state)
        
        # Pass the action to the environment
        state_new, reward, done,_ = env.step(action)
        state_new = np.eye(N=1, M=16, k=state_new, dtype=int)[0]

        # Add this new experience to the agents memory
        agent.remember(state, action, reward, state_new, done)
   
        # Iterate variables and check if the episode is done (in terminal state)
        episode_reward += reward
        step += 1       
        state = state_new
        if done:
            print("Episode {}/{} - {} steps - {} reward".format(episode, num_episodes, step, episode_reward))
            break
    
    if len(agent.memory) > batch_size:
        agent.replay(batch_size)      
        





Episode 0/10 - 6 steps - 0.0 reward


ValueError: Error when checking input: expected dense_1_input to have shape (16,) but got array with shape (1,)

In [4]:
batch_size = 4
state, action, reward, next_state, done = zip(*random.sample(agent.memory,batch_size))

future_q = agent.model.predict(np.array(next_state))
target_out = agent.model.predict(np.array(state))
print(target_out)
for x in range(0,batch_size):
    if not done[x]:
        target = reward[x] + agent.gamma * np.amax(future_q[x][0])
    else:
        target = reward[x]
    
    target_out[x][action[x]] = target
print(target_out)


NameError: name 'agent' is not defined

In [129]:
print(done)
print(future_q)

(False, False, False, True)
[[ 0.04340317  0.24338077 -0.13712645  0.00522141]
 [-0.10781736  0.14788064 -0.06163432  0.20870192]
 [ 0.12785308  0.13542403  0.11849722  0.14539072]
 [ 0.03470756 -0.01242799  0.03000678  0.27064362]]


In [112]:
state = np.array([[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])
agent.model.predict(state)

array([[-0.01041937, -0.22935791,  0.10132787, -0.17830871]],
      dtype=float32)

In [83]:
state, action, reward, next_state, done = zip(*random.sample(agent.memory,6))

In [84]:
np.array(state)

array([[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [85]:
agent.model.predict(np.array(state))

array([[ 0.05996166, -0.18592815, -0.26443177,  0.19104314],
       [-0.09735326, -0.12600815, -0.2244632 ,  0.03633647],
       [-0.09735326, -0.12600815, -0.2244632 ,  0.03633647],
       [-0.09735326, -0.12600815, -0.2244632 ,  0.03633647],
       [ 0.05996166, -0.18592815, -0.26443177,  0.19104314],
       [-0.07036515, -0.03292663,  0.09776071,  0.18202789]],
      dtype=float32)

In [118]:
# Similar to the Q-learning table, we now test the DQN removing all randomness

num_episodes = 100

# Create a list that stores the episode_reward and step count at each episode
reward_list = []
step_list = []

for episode in range(num_episodes):
    # Reset the env
    pos = env.reset()
    state = np.eye(N=1, M=16, k=pos, dtype=int)[0] # We use the numpy.eye function to create the one-hot vector
    step = 0
    episode_reward = 0
    # We limit the amount of steps to avoid any infinite or long episodes
    while step < 99:
        action = agent.act(state)
        
        # Pass the action to the environment
        state_new, reward, done,_ = env.step(action)
        state_new = np.eye(N=1, M=16, k=state_new, dtype=int)[0]
            
        # Iterate variables and check if the episode is done (in terminal state)
        episode_reward += reward
        step += 1       
        state = state_new
        if done:
            print("Episode {}/{} - {} steps - {} reward".format(episode, num_episodes, step, episode_reward))
            break
    
    # Add the episode rewards and steps to a list so we can review performance later on
    reward_list.append(episode_reward)
    step_list.append(step)       

Episode 0/100 - 18 steps - 0.0 reward
Episode 1/100 - 2 steps - 0.0 reward
Episode 2/100 - 2 steps - 0.0 reward
Episode 3/100 - 3 steps - 0.0 reward
Episode 4/100 - 7 steps - 0.0 reward
Episode 5/100 - 9 steps - 0.0 reward
Episode 6/100 - 35 steps - 0.0 reward
Episode 7/100 - 2 steps - 0.0 reward
Episode 8/100 - 9 steps - 0.0 reward
Episode 9/100 - 9 steps - 0.0 reward
Episode 10/100 - 2 steps - 0.0 reward
Episode 11/100 - 7 steps - 0.0 reward
Episode 12/100 - 10 steps - 0.0 reward
Episode 13/100 - 6 steps - 0.0 reward
Episode 14/100 - 10 steps - 0.0 reward
Episode 15/100 - 12 steps - 0.0 reward
Episode 16/100 - 2 steps - 0.0 reward
Episode 17/100 - 2 steps - 0.0 reward
Episode 18/100 - 8 steps - 0.0 reward
Episode 19/100 - 4 steps - 0.0 reward
Episode 20/100 - 6 steps - 0.0 reward
Episode 21/100 - 6 steps - 0.0 reward
Episode 22/100 - 11 steps - 0.0 reward
Episode 23/100 - 2 steps - 0.0 reward
Episode 24/100 - 12 steps - 0.0 reward
Episode 25/100 - 12 steps - 0.0 reward
Episode 26/100

In [119]:
print ("Average score over time: {}".format(str(sum(reward_list)/num_episodes)))
print ("Average steps over time: {}".format(str(sum(step_list)/num_episodes)))

# Average score over time: 0.72
# Average steps over time: 34.17

Average score over time: 0.0
Average steps over time: 7.64
